In [1]:
import os
import nibabel as nib
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import albumentations as A


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import os
dataset_path = "/kaggle/input/camus-datasert"
files = os.listdir(dataset_path)
print(files)  # To verify the dataset is available


['database_nifti']


In [3]:
# Set fixed image size
IMG_SIZE = 256

# Define dataset root path
DATASET_ROOT = "/kaggle/input/camus-datasert/database_nifti"  # Change this to your actual dataset folder

# Define the number of augmented samples per image
AUGMENTATIONS_PER_IMAGE = 2  # Adjust this to reach your desired dataset size

# Define a more aggressive augmentation pipeline
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, p=0.5),
    A.RandomBrightnessContrast(contrast_limit=0.2, brightness_limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
    A.OpticalDistortion(distort_limit=0.3, shift_limit=0.1, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
])

def load_nii(filepath):
    """Load a .nii.gz file and return the image as a NumPy array."""
    img = nib.load(filepath).get_fdata()
    img = np.squeeze(img)  # Remove single dimensions
    return img

def preprocess_image(image):
    """Resize and normalize image."""
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize
    image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-7)  # Normalize
    return image

# Lists to store images & masks
images, masks = [], []

# Check if augmented dataset already exists
if os.path.exists("augmented_images.npy") and os.path.exists("augmented_masks.npy"):
    print("Loading augmented dataset from disk...")
    images = np.load("augmented_images.npy")
    masks = np.load("augmented_masks.npy")
else:
    print("Augmenting dataset...")
    # Loop through patient folders
    for patient_folder in os.listdir(DATASET_ROOT):
        patient_path = os.path.join(DATASET_ROOT, patient_folder)

        # Ensure it's a folder
        if os.path.isdir(patient_path):
            # Get all .nii.gz files in the patient folder
            nii_files = sorted([f for f in os.listdir(patient_path) if f.endswith(".nii")])

            for nii_file in nii_files:
                if "_gt.nii" in nii_file:  # This is a mask file, skip it in this loop
                    continue
                
                # Define corresponding mask filename
                mask_file = nii_file.replace(".nii", "_gt.nii")

                img_path = os.path.join(patient_path, nii_file)
                mask_path = os.path.join(patient_path, mask_file)

                # Check if both image & mask exist
                if os.path.exists(mask_path):
                    try:
                        # Load image and mask
                        image = load_nii(img_path)
                        mask = load_nii(mask_path)

                        # Ensure image and mask are 2D
                        if image.ndim > 2:
                            image = image[..., 0]  # Take the first slice if 3D
                        if mask.ndim > 2:
                            mask = mask[..., 0]  # Take the first slice if 3D

                        # Resize and normalize
                        image = preprocess_image(image)
                        mask = preprocess_image(mask)

                        # Append original image and mask
                        images.append(image)
                        masks.append(mask)

                        # Generate augmented samples
                        for _ in range(AUGMENTATIONS_PER_IMAGE):
                            augmented = augmentation(image=image, mask=mask)
                            image_aug, mask_aug = augmented["image"], augmented["mask"]
                            images.append(image_aug)
                            masks.append(mask_aug)

                    except Exception as e:
                        print(f"❌ Error loading {nii_file}: {e}")

    # Convert to NumPy arrays
    images = np.stack(images).astype(np.float32)  # Ensure all are the same shape
    masks = np.stack(masks).astype(np.float32)    # Ensure all are the same shape

    # Add channel dimension (for compatibility with U-Net)
    images = np.expand_dims(images, axis=-1)  # Shape: (N, IMG_SIZE, IMG_SIZE, 1)
    masks = np.expand_dims(masks, axis=-1)    # Shape: (N, IMG_SIZE, IMG_SIZE, 1)

    # Save the augmented dataset
    np.save("augmented_images.npy", images)
    np.save("augmented_masks.npy", masks)

# Split dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(images, masks, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Print dataset sizes
print(f"✅ Training: {X_train.shape[0]} images")
print(f"✅ Validation: {X_val.shape[0]} images")
print(f"✅ Testing: {X_test.shape[0]} images")

Augmenting dataset...
✅ Training: 7200 images
✅ Validation: 900 images
✅ Testing: 900 images


In [4]:
import numpy as np

# Load the augmented dataset
images = np.load("augmented_images.npy")
masks = np.load("augmented_masks.npy")

# Print dataset shape
print(f"✅ Loaded augmented dataset:")
print(f"Images shape: {images.shape}")
print(f"Masks shape: {masks.shape}")

✅ Loaded augmented dataset:
Images shape: (9000, 256, 256, 1)
Masks shape: (9000, 256, 256, 1)


In [5]:
# Split dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(images, masks, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [4]:
# Define Dice Loss
def dice_loss(y_true, y_pred, smooth=1e-7):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1.0 - dice

# Define Combined Loss (Dice + Binary Cross-Entropy)
def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

In [5]:

# Define Dice Coefficient Metric
def dice_coefficient(y_true, y_pred, smooth=1e-7):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return dice

# Define IoU Metric
def iou(y_true, y_pred, smooth=1e-7):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

In [6]:
# Define Classic U-Net
def classic_unet(input_shape=(256, 256, 1)):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
from tensorflow.keras.models import load_model

# Load the best model
model = load_model("/kaggle/input/model_v1/keras/default/1/best_model.keras", custom_objects={
    "combined_loss": combined_loss,
    "dice_coefficient": dice_coefficient,
    "iou": iou
})

In [9]:
# Compile the Model
model = classic_unet(input_shape=(256, 256, 1))
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=combined_loss,
              metrics=[dice_coefficient, iou])

# Learning Rate Scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7,verbose=1)

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="/kaggle/working//best_modelV2.keras",  # Path to save the best model
    monitor="val_dice_coefficient",  # Metric to monitor
    mode="max",  # Save the model with the maximum validation Dice Coefficient
    save_best_only=True,  # Only save the best model
    verbose=1  # Print a message when the model is saved
)

# Learning Rate Scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7,verbose=1)

In [12]:
# Train the Model
history = model.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    batch_size=16,
                    epochs=10,
                    callbacks=[reduce_lr,checkpoint_callback])

Epoch 1/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 984ms/step - dice_coefficient: 0.6525 - iou: 0.4845 - loss: 0.5506
Epoch 1: val_dice_coefficient improved from -inf to 0.64467, saving model to /kaggle/working//best_modelV2.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 580s 1s/step - dice_coefficient: 0.6525 - iou: 0.4845 - loss: 0.5506 - val_dice_coefficient: 0.6447 - val_iou: 0.4758 - val_loss: 0.5514 - learning_rate: 1.0000e-04
Epoch 2/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 987ms/step - dice_coefficient: 0.6644 - iou: 0.4976 - loss: 0.5300
Epoch 2: val_dice_coefficient improved from 0.64467 to 0.65651, saving model to /kaggle/working//best_modelV2.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 461s 1s/step - dice_coefficient: 0.6644 - iou: 0.4976 - loss: 0.5300 - val_dice_coefficient: 0.6565 - val_iou: 0.4889 - val_loss: 0.5464 - learning_rate: 1.0000e-04
Epoch 3/10
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 988ms/step - dice_coefficient: 0.6710 - iou: 0.5051 - loss: 0.5190
Epoch 3: val_dice_coefficient improved from 0.65651 t

In [11]:

# Train the Model
history = model.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    batch_size=16,
                    epochs=15,
                    callbacks=[reduce_lr,checkpoint_callback])



Epoch 1/15
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 986ms/step - dice_coefficient: 0.3684 - iou: 0.2302 - loss: 1.0195
Epoch 1: val_dice_coefficient improved from -inf to 0.51086, saving model to /kaggle/working/best_model.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 583s 1s/step - dice_coefficient: 0.3685 - iou: 0.2303 - loss: 1.0192 - val_dice_coefficient: 0.5109 - val_iou: 0.3433 - val_loss: 0.7801 - learning_rate: 1.0000e-04
Epoch 2/15
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 985ms/step - dice_coefficient: 0.5403 - iou: 0.3707 - loss: 0.7383
Epoch 2: val_dice_coefficient improved from 0.51086 to 0.55061, saving model to /kaggle/working/best_model.keras
450/450 ━━━━━━━━━━━━━━━━━━━━ 460s 1s/step - dice_coefficient: 0.5403 - iou: 0.3707 - loss: 0.7383 - val_dice_coefficient: 0.5506 - val_iou: 0.3801 - val_loss: 0.7073 - learning_rate: 1.0000e-04
Epoch 3/15
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 984ms/step - dice_coefficient: 0.5799 - iou: 0.4088 - loss: 0.6720
Epoch 3: val_dice_coefficient improved from 0.55061 to 0.59

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("my_model.keras", custom_objects={
    "combined_loss": combined_loss,
    "dice_coefficient": dice_coefficient,
    "iou": iou
})

In [ ]:
# Save the model
model.save('/kaggle/working/my_model2.keras')

In [ ]:
# Save the entire model
model.save("my_model.keras")  # or use .h5 for HDF5 format

In [ ]:
# Save the entire model
model.save("my_model.h5")  # or use .h5 for HDF5 format

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("my_model.keras", custom_objects={
    "combined_loss": combined_loss,
    "dice_coefficient": dice_coefficient,
    "iou": iou
})

In [ ]:
# Continue training for another 30 epochs
history_continued = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=60,  # Total epochs (30 new + 30 previous)
    initial_epoch=30,  # Start from epoch 30
    callbacks=[checkpoint_callback, reduce_lr]  # Include the checkpoint callback
)